#### 1° String de conexão (ajuste conforme seu ambiente)

In [40]:
from sqlalchemy import create_engine, text

server = 'DESKTOP-F0DI39E'
database = 'master'  # criaremos o banco a partir do master
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)

#### 2° (OPCIONAL) Criar a base de dados via Python

In [ ]:
# from sqlalchemy import Engine, text


# with Engine.connect() as conn:
#     conn.execute(text("""
#         IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'BI_Projeto')
#         CREATE DATABASE BI_Projeto;
#     """))
#     conn.commit()

#### 3° Conectar na base criada

In [41]:
database = 'ProjetoBI'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)

#### 4° Criação das tabelas dbo.Cliente

In [42]:

create_table_sql = """
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Ourobase_ProjetoBI'
)
BEGIN
    CREATE TABLE dbo.Ourobase_ProjetoBI (
        
    -- Cliente / Usuário
    
    int_CodCliente INT IDENTITY(1,1),
    str_Cliente VARCHAR(150) NOT NULL,
    str_Usuario VARCHAR(150) NULL,
    str_Email VARCHAR(150) NOT NULL,
    bit_StatusAtivo BIT NOT NULL,
    str_DescrInativo VARCHAR(255) NULL,

    -- Servidor / Infra
    
    int_CodTipoAcessoServidor INT NOT NULL,
    str_TipoAcessoServidor VARCHAR(100) NOT NULL,
    str_NomeServidor VARCHAR(150) NOT NULL,
    str_NomeDataBase VARCHAR(150) NOT NULL,

    -- Projeto BI
    
    int_CodNomeProjetoBI INT NOT NULL,
    str_NomeProjetoBI VARCHAR(150) NOT NULL,
    bit_PossuiAplicativo BIT NOT NULL,
    str_TipoLicenca VARCHAR(100) NOT NULL,
    bit_PossuiRegraNegocio BIT NULL,
    str_DescrRegraRLSBI VARCHAR(255) NULL,
    int_QtdeAtualizacoes INT NOT NULL,
    str_AtualizacaoAgendada VARCHAR(100) NULL,
    bit_PossuiBIPersonalizado BIT NOT NULL,
    str_DescrBIPersonalizado VARCHAR(255) NULL,

    -- Melhorias
    
    int_CodMelhoriasBI INT NOT NULL,
    bit_MelhoriasBI BIT NOT NULL,
    str_DescrMelhoriasBI VARCHAR(255) NULL,
    dta_DataMelhoria DATE NULL,

    -- Pendências
    
    int_CodPendencia INT NULL,
    dta_DataInicioPendencia DATE NULL,
    dta_DataFimPendencia DATE NULL,

    -- Atendimentos
    
    int_CodAtendimento INT NULL,
    dta_DataInicioAtendimento DATE NULL,
    dta_DataFimAtendimento DATE NULL,

    -- Controle
    
    int_CodVersionamentoProjeto INT NOT NULL,
    str_VersionamentoProjeto VARCHAR(50) NOT NULL,
    dta_DataCadastro DATE NULL

    );
END
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

#### 5°Inserir dados via Python dbo.Cliente

In [ ]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-F0DI39E;"
    "DATABASE=ProjetoBI;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

# Truncate
cursor.execute("TRUNCATE TABLE dbo.Ourobase_ProjetoBI")
conn.commit()

# Load Excel
df = pd.read_excel("BaseProjetoDesnormalizada.xlsx", sheet_name="Geral")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO dbo.Ourobase_ProjetoBI (str_Cliente, str_Usuario, str_Email, bit_StatusAtivo, str_DescrInativo, str_TipoAcessoServidor,
        str_NomeServidor, str_NomeDataBase,	str_NomeProjetoBI, bit_PossuiAplicativo, str_TipoLicenca, bit_PossuiRegraNegocio, str_DescrRegraRLSBI,
        str_AtualizacaoAgendada, bit_PossuiBIPersonalizado, str_DescrBIPersonalizado,bit_MelhoriasBI, str_DescrMelhoriasBI, dta_DataMelhoria,
        dta_DataInicioPendencia, dta_DataFimPendencia, dta_DataInicioAtendimento, dta_DataFimAtendimento, str_VersionamentoProjeto, dta_DataCadastro)
        VALUES (?)
    """, row['str_Cliente'], ['str_Usuario'], ['str_Email'], ['bit_StatusAtivo'], ['str_DescrInativo'], ['str_TipoAcessoServidor'], ['str_NomeServidor']
            ['str_NomeDataBase'], ['str_NomeDataBase'], ['str_NomeProjetoBI'],['bit_PossuiAplicativo'], ['str_TipoLicenca'], ['bit_PossuiRegraNegocio'],
            ['str_DescrRegraRLSBI'],['str_AtualizacaoAgendada'], ['bit_PossuiBIPersonalizado'],['str_DescrBIPersonalizado'], ['bit_MelhoriasBI'],
            ['str_DescrMelhoriasBI'], ['dta_DataMelhoria'],['dta_DataInicioPendencia'],['dta_DataFimPendencia'],['dta_DataInicioAtendimento'],
            ['dta_DataFimAtendimento'],['str_VersionamentoProjeto'], ['dta_DataCadastro'])

conn.commit()
cursor.close()
conn.close()


Error: ('21S01', '[21S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Existem mais colunas na instrução INSERT do que valores especificados na cláusula VALUES. O número de valores da cláusula VALUES deve corresponder ao número de colunas especificado na instrução INSERT. (109) (SQLExecDirectW); [21S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Não foi possível preparar uma ou mais instruções. (8180)')

#### 6° (Opcional) Consultar os dados

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dbo.Cliente"))
    for row in result:
        print(row)

### 7° OPICIONAL Atualizar com SQLAlchemy (caso você esteja usando)

In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(
    "mssql+pyodbc://@localhost/BI_Projeto?driver=ODBC+Driver+17+for+SQL+Server"
)

with engine.connect() as conn:
    result = conn.execute(
        text("""
            UPDATE dbo.Cliente
            SET str_Cliente = :nome
            WHERE id_Cliente = :id
        """),
        {"nome": "001 - singular medicamentos", "id": 2}
    )
    conn.commit()